In [ ]:
# 1. Install dependencies
!pip install -q torch torchvision numpy pillow scikit-image opencv-python tqdm requests matplotlib earthengine-api kaggle

In [ ]:
# 2. Setup path
import sys
sys.path.insert(0, '.')

# Verify
from src.config import SR_SCALE
print(f"Project loaded. SR scale: {SR_SCALE}x")

In [ ]:
# 3. GEE Auth
import ee
ee.Authenticate()
ee.Initialize()
print("GEE authenticated")

In [ ]:
# 4. Kaggle Auth - upload kaggle.json when prompted
from google.colab import files
print("Upload kaggle.json:")
files.upload()
!mkdir -p ~/.kaggle && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json
print("Kaggle configured")

In [ ]:
# 5. Download WorldStrat
!kaggle datasets download -d julienco/worldstrat -p datasets/worldstrat --unzip

In [ ]:
# 6. Train (3 epochs, ~15 min)
from src.training.finetune import finetune_swinir
from src.data.worldstrat_loader import WorldStratDataset
from pathlib import Path

train_ds = WorldStratDataset(root_dir=Path('datasets/worldstrat'), split='train', max_samples=200)
val_ds = WorldStratDataset(root_dir=Path('datasets/worldstrat'), split='validation', max_samples=40)

results = finetune_swinir(train_dataset=train_ds, val_dataset=val_ds, batch_size=4, max_epochs=3, device='cuda')
print(f"Done! Best loss: {results['best_loss']:.4f}")

In [ ]:
# 7. Fetch tile & run SR
from src.data.gee_fetcher import GEEFetcher
from src.inference.pipeline import SuperResolutionPipeline
import matplotlib.pyplot as plt

tile = GEEFetcher(authenticate=True).fetch_tile('delhi', tile_size=256)
results = SuperResolutionPipeline(device='cuda').run(tile)

fig, ax = plt.subplots(1, 3, figsize=(15, 5))
ax[0].imshow(results['lr']); ax[0].set_title('LR (10m)')
ax[1].imshow(results['bicubic']); ax[1].set_title('Bicubic 4x')
ax[2].imshow(results['sr']); ax[2].set_title('SwinIR 4x (2.5m)')
plt.savefig('result.png')
plt.show()
print(f"Input: {results['lr'].shape} -> Output: {results['sr'].shape}")